# Wrangle and Analyze Data of a Twitter Account


## Table of Contents
- [1. Introduction](#intro)
- [2. Gather data](#gather)
- [3. Assess data](#assess)
- [4. Clean data](#clean)
- [5. Store](#store)


<a id='gather'></a>
## 1. Introduction

This project is an analysis of correlation between the Covid-19 cases and the political environment of different countries. Goal is to find answers or at least indicators to questions like: 
- Did the countries which had more success in containing the amount of Covid-19 cases something in common? 
- Is there a correlation in  Gross domestic product, Human Development Index or political ideology with the amount of Covid-19 cases of the country.

Main goal of this project is to generate a comprehensive exploratory and explanatory data analysis of the gathered data. The data analysis process is distributed over three ipynb-files: gather_clean_Covid19.ipynb, exploration_Covid19.ipynb and slide_deck_Covid19.ipynb.

Firstly, as part of gather_clean_Covid19.ipynb data is gathered from different sources: The Covid-19 data of this project is retrieved via programmatically downloaded csv-files from the GitHub repository [Covid-19](https://github.com/CSSEGISandData/COVID-19) and additional data about countries is retrieved via the wikipedia API. Secondly, the data from the different sources is visually and programmatically assessed to be cleaned.
The exploratory and explanatory data analysis of the gathered data is performed in exploration_Covid19.ipynb. Finally the findings are presented in slide_deck_Covid19.ipynb.

In [1280]:
# Import necessary libraries
import numpy as np
import pandas as pd
from datetime import date
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import os # to work with local directory
import re
import wptools
import json # to create json file from python dictionary
import time # for timer 
sns.set()

<a id='intro'></a>
## 2. Gather data

####  Data is gathered from three different sources of data as described in steps below:

1. Fatality, confirmed cases, recovered cases and data by country is retrieved via programmatically downloaded csv-files from the GitHub repository [Covid-19](https://github.com/CSSEGISandData/COVID-19).
2. Additional data is retrieved via the wptools API from different wikipedia articles.

### a. Read data from programmatically download csv-file

In [1281]:
# Gather data from John Hopkins GitHub 
df_JHU_Fatality = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
df_JHU_Confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_JHU_Recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
df_JHU_Countries = pd.read_csv('https://raw.githubusercontent.com/RRighart/covid-19/master/countries.csv')

In [1282]:
df_OWID_Covid = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')
df_OWID_Testing = pd.read_csv('https://covid.ourworldindata.org/data/testing/covid-testing-latest-data-source-details.csv')
df_OWID_Countries = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/locations.csv')

### b. Read data from local datasets

Data downloaded manually from different databases, [European statistical database](https://ec.europa.eu/eurostat/data/database), [Wikipedia table on intensive care units](https://en.wikipedia.org/wiki/List_of_countries_by_hospital_beds) and [United Nations database](https://data.un.org):

In [1283]:
df_ESTAT_census = pd.read_csv('inputData/Eurostat_census_2001.csv')
df_WIKI_ICU = pd.read_csv('inputData/Wikipedia_ICU.csv')
df_UN_births = pd.read_csv('inputData/UNdata_birthsByMonth.csv')
df_UN_deaths = pd.read_csv('inputData/UNdata_deathsByMonth.csv')

### c. Query additional information for countries via wikipedia API

Additional Information
- Leader Gender
- Ideology of Leading Party
- Amount of Intensive Care Beds
- Gross domestic product per capita
- Human Development Index

In [1284]:
# Query for every tweet id in enhanced twitter archive and save tweet-information in json-format to 'tweet_json.txt'
'''             
country_jsons = {}
county_id_errors = []
start = time.time()
count = 0


with open('country_json.txt', 'w') as outfile:
    
    for country in df_JHU_Countries['Country/Region']:
        count +=1
        try:
            # Query API for data of wikipedia article
            article = wptools.page(country).get_parse()
            infobox = article.data['infobox']
            # Measure elapsed time
            mid_s = time.time()
            # Print id and time elapsed
            print(str(count) + str(mid_s - start) )
            # Write json of tweet to 'tweet_json.txt'
            json.dump(infobox, outfile)
            # New line
            outfile.write("\n")

        # Not best practice to catch all exceptions but fine for this short script
        except Exception as error:
            mid_f = time.time()
            print(str(count) + str(mid_f - start) + str(error))
            # Gather ids of id's without status
            tweet_id_errors.append([count, str(tweet_id)])
            
    end = time.time()
    print(end - start)
    
    '''

'             \ncountry_jsons = {}\ncounty_id_errors = []\nstart = time.time()\ncount = 0\n\n\nwith open(\'country_json.txt\', \'w\') as outfile:\n    \n    for country in df_JHU_Countries[\'Country/Region\']:\n        count +=1\n        try:\n            # Query API for data of wikipedia article\n            article = wptools.page(country).get_parse()\n            infobox = article.data[\'infobox\']\n            # Measure elapsed time\n            mid_s = time.time()\n            # Print id and time elapsed\n            print(str(count) + str(mid_s - start) )\n            # Write json of tweet to \'tweet_json.txt\'\n            json.dump(infobox, outfile)\n            # New line\n            outfile.write("\n")\n\n        # Not best practice to catch all exceptions but fine for this short script\n        except Exception as error:\n            mid_f = time.time()\n            print(str(count) + str(mid_f - start) + str(error))\n            # Gather ids of id\'s without status\n       

In [1285]:
'''
so = wptools.page('Germany').get_parse()
infobox = so.data['infobox']
print(infobox)
'''

"\nso = wptools.page('Germany').get_parse()\ninfobox = so.data['infobox']\nprint(infobox)\n"

<a id='assess'></a>
## 3. Assess data

After gathering each of the above pieces of data, they are assessed visually and programmatically for quality and tidiness issues. Requirements to be met:

- Quality requirements:
    - Completeness: All necessary records in dataframes, no specific rows, columns or cells missing.
    - Validity: No records available, that do not conform schema.
    - Accuracy: No wrong data, that is valid.
    - Consistency: No data, that is valid and accurate, but referred to in multiple correct ways.
- Tidiniss requirements (as defined by Hadley Wickham):
    - each variable is a column
    - each observation is a row
    - each type of observational unit is a table.

### a. Visual assessment

In [1286]:
# Check layout of df_JHU_Countries vsiually
df_JHU_Countries.sample(n=5)

,Unnamed: 0,Country/Region,inhabitants,area
8,8,United Kingdom,67803450,242495
4,4,Netherlands,17123478,41198
7,7,Spain,46767543,498511
23,23,Finland,5540720,303890
22,22,Denmark,5792202,42430


In [1287]:
# Check layout of df_JHU_Fatality vsiually
df_JHU_Fatality.sample(n=5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
97,NaN,Ecuador,-1.8312,-78.1834,0,0,0,0,0,0,...,1654,1704,1717,2127,2145,2327,2334,2338,2594,2688
21,NaN,Barbados,13.1939,-59.5432,0,0,0,0,0,0,...,7,7,7,7,7,7,7,7,7,7
122,NaN,Greece,39.0742,21.8243,0,0,0,0,0,0,...,148,150,151,151,151,152,155,156,160,162
230,NaN,Zimbabwe,-20.0000,30.0000,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,4,4
76,Sichuan,China,30.6171,102.7103,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3


In [1288]:
# Check layout of df_JHU_Confirmed vsiually
df_JHU_Confirmed.sample(n=5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
248,Anguilla,United Kingdom,18.2206,-63.0686,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3
188,NaN,Rwanda,-1.9403,29.8739,0,0,0,0,0,0,...,271,273,280,284,285,286,287,287,287,289
94,NaN,Denmark,56.2639,9.5018,0,0,0,0,0,0,...,10083,10218,10319,10429,10513,10591,10667,10713,10791,10858
106,NaN,Finland,64.0000,26.0000,0,0,0,0,0,0,...,5673,5738,5880,5962,5984,6003,6054,6145,6228,6286
60,Henan,China,33.8820,113.6140,5,5,9,32,83,128,...,1276,1276,1276,1276,1276,1276,1276,1276,1276,1276


In [1289]:
# Check layout of df_JHU_Recovered vsiually
df_JHU_Recovered.sample(n=5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
211,NaN,Trinidad and Tobago,10.6918,-61.2225,0,0,0,0,0,0,...,103,103,104,107,107,107,107,107,107,107
7,NaN,Armenia,40.0691,45.0382,0,0,0,0,0,0,...,1185,1218,1267,1325,1359,1430,1500,1572,1666,1791
111,NaN,Georgia,42.3154,43.3569,0,0,0,0,0,0,...,275,288,297,309,317,349,372,383,393,419
141,NaN,Lebanon,33.8547,35.8623,0,0,0,0,0,0,...,220,223,234,234,234,234,236,236,246,247
204,NaN,Switzerland,46.8182,8.2275,0,0,0,0,0,0,...,25900,26100,26400,26600,26800,26800,27100,27100,27100,27400


In [1290]:
# Check layout of df_OWID_Covid vsiually
df_OWID_Covid.sample(n=5)

# df_OWID_Covidchange 'location' to 'country'
# df_OWID_Covid create df_OWID_Countries with 'iso_code', 'location', 'population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'diabetes_prevalence', 'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_100k'
# df_OWID_Covid merge it to df_country

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
2490,BRB,Barbados,2020-04-25,76,0,6,0,264.466,0.000,20.879,...,14.952,9.473,16978.068,NaN,170.050,13.57,1.9,14.5,88.469,5.80
936,AUS,Australia,2020-04-22,6647,22,74,3,260.668,0.863,2.902,...,15.504,10.129,44648.710,0.5,107.791,5.07,13.0,16.5,NaN,3.84
3836,CUW,Curacao,2020-05-09,16,0,1,0,97.502,0.000,6.094,...,16.367,10.068,NaN,NaN,NaN,11.62,NaN,NaN,NaN,NaN
2741,CAF,Central African Republic,2020-05-03,72,0,0,0,14.908,0.000,0.000,...,3.655,2.251,661.240,NaN,435.727,6.10,NaN,NaN,16.603,1.00
5916,GAB,Gabon,2020-04-01,16,9,1,0,7.189,4.044,0.449,...,4.450,2.976,16562.413,3.4,259.967,7.20,NaN,NaN,NaN,6.30


In [1291]:
# Check layout of df_OWID_Testing vsiually
df_OWID_Testing.sample(n=5)

# df_OWID_Testing drop columns 'source URL', 'Source label', 'Notes', 'Number of observations', 'Daily change in cumulative total', 'Daily change in cumulative total per thousand', '3-day rolling mean daily change', '3-day rolling mean daily change per thousand', '7-day rolling mean daily change', '7-day rolling mean daily change per thousand','General source label', 'General source URL', 'Short description', 'Detailed description'
# df_OWID_Testing Either cut per regex country name from 'Entity' and rename country or join country name from other df

,ISO code,Entity,Date,Source URL,Source label,Notes,Number of observations,Cumulative total,Cumulative total per thousand,Daily change in cumulative total,Daily change in cumulative total per thousand,3-day rolling mean daily change,3-day rolling mean daily change per thousand,7-day rolling mean daily change,7-day rolling mean daily change per thousand,General source label,General source URL,Short description,Detailed description
52,NZL,New Zealand - tests performed,2020-05-15,https://www.health.govt.nz/our-work/diseases-a...,Ministry of Health,NaN,68,223937,46.438,7150.0,1.483,6964.000,1.444,5842.571,1.212,New Zealand Ministry of Health,https://www.health.govt.nz/our-work/diseases-a...,The number of tests performed.,The New Zealand Ministry of Health provide a t...
86,GBR,United Kingdom - tests performed,2020-05-16,https://web.archive.org/web/20200516192746/htt...,Public Health England/Department of Health and...,NaN,41,2489563,36.673,136486.0,2.011,132111.333,1.946,108859.571,1.604,Public Health England/Department of Health and...,https://www.gov.uk/guidance/coronavirus-covid-...,"The number of tests performed, including tests...",A time series is not yet released. The figures...
53,NGA,Nigeria - samples tested,2020-05-16,https://web.archive.org/web/20200516120106/htt...,Nigeria Centre for Disease Control,NaN,19,32942,0.160,NaN,NaN,NaN,NaN,NaN,NaN,Nigeria Centre for Disease Control,https://ncdc.gov.ng/diseases/sitreps/?cat=14&n...,The number of samples tested.,"The Nigeria Centre for Disease Control, on the..."
1,AUS,Australia - tests performed,2020-05-16,https://www.health.gov.au/sites/default/files/...,Australian Government Department of Health,NaN,44,1015652,39.830,31836.0,1.248,35542.333,1.394,31456.571,1.234,Australian Government Department of Health,https://www.health.gov.au/news/health-alerts/n...,The number of tests performed.,We construct a time series of the cumulative n...
16,CZE,Czech Republic - tests performed,2020-05-15,https://onemocneni-aktualne.mzcr.cz/api/v1/cov...,Ministry of Health,NaN,104,348849,32.575,6601.0,0.616,7053.667,0.658,6501.143,0.607,Ministry of Health,https://onemocneni-aktualne.mzcr.cz/api/v1/cov...,The number of tests performed.,The Ministry of Health publishes a data set co...


In [1292]:
# Check layout of df_OWID_Countries vsiually
df_OWID_Countries.sample(n=5)

# df_OWID_Countries convert datatype population to integer
# df_OWID_Countries drop 'countriesAndTerritories', 'population_year'

,countriesAndTerritories,location,continent,population_year,population
64,Eswatini,Swaziland,Africa,2020.0,1160164.0
149,Panama,Panama,North America,2020.0,4314768.0
8,Armenia,Armenia,Asia,2020.0,2963234.0
169,Seychelles,Seychelles,Africa,2020.0,98340.0
84,Guinea,Guinea,Africa,2020.0,13132792.0


In [1293]:
# Check layout of df_ESTAT_census vsiually
df_ESTAT_census.sample(n=5)

# df_ESTAT_census make columns from values in n_person
# df_ESTAT_census replace 'Germany (until 1990 former territory of the FRG)' with 'Germany'
# df_ESTAT_census drop country 'Bulgaria'
# df_ESTAT_census drop '4 persons', '5 persons', '6 persons or more'

,N_PERSON,GEO,TIME,AGE,HHCOMP,UNIT,Value,Flag and Footnotes
0,Total,Bulgaria,2001,65 years or over,Total,Number,"2,921,887",NaN
202,5 persons,Luxembourg,2001,65 years or over,Total,Number,:,NaN
30,No persons,Germany (until 1990 former territory of the FRG),2001,65 years or over,Total,Number,"27,061,600",NaN
13,Total,Luxembourg,2001,65 years or over,Total,Number,"171,953",NaN
101,2 persons,Slovenia,2001,65 years or over,Total,Number,"59,333",NaN


In [1294]:
df_ESTAT_census.AGE.value_counts()

65 years or over    243
Name: AGE, dtype: int64

In [1295]:
# Check layout of df_WIKI_ICU vsiually
df_WIKI_ICU.sample(n=5)

,countryname,continent,hospital_beds_per_1000_people,occupancy,ICU-CCB_beds_per_1000_people,ventilators
20,Australia,Oceania,3.84,NaN,9.1,"1,314"
14,Estonia,Europe,4.69,70.4,14.6,NaN
37,Chile,South America,2.11,79.1,NaN,NaN
0,Japan,Asia,13.05,75.5,7.3,"32,586"
23,Netherlands,Europe,3.32,65.4,6.4,NaN


In [1296]:
df_UN_births.sample(n=5)
# Drop columns 'Area', 'Record Type', 'Reliability', 'Value Footnotes', 'Source Year'
# change datatype of columns  'Value' to integer
# Merge df_UN_births and df_UN_deaths on Year


,Country or Area,Year,Area,Month,Record Type,Reliability,Source Year,Value,Value Footnotes
5038,Kazakhstan,2011,Total,August,Data tabulated by year of occurrence,"Final figure, complete",2012.0,33573.0,NaN
9908,Turks and Caicos Islands,2014,Total,February,Data tabulated by year of occurrence,"Final figure, complete",2017.0,36.0,NaN
4391,Iceland,2011,Total,September,Data tabulated by year of occurrence,"Final figure, complete",2012.0,376.0,NaN
7970,Republic of Korea,2014,Total,November,Data tabulated by year of occurrence,"Final figure, complete",2019.0,32379.0,33
4984,Kazakhstan,2015,Total,October,Data tabulated by year of occurrence,"Final figure, complete",2018.0,33464.0,NaN


In [1297]:
df_UN_births.Area.value_counts()

Total    10373
Name: Area, dtype: int64

In [1298]:
df_UN_deaths.sample(n=5)

,Country or Area,Year,Area,Month,Record Type,Reliability,Source Year,Value,Value Footnotes
5332,Luxembourg,2011,Total,July,Data tabulated by year of occurrence,"Final figure, complete",2012.0,316.0,NaN
3385,Georgia,2011,Total,June,Data tabulated by year of occurrence,"Final figure, complete",2012.0,3844.0,1
9397,United States of America,2015,Total,September,Data tabulated by year of occurrence,"Final figure, complete",2017.0,209905.0,NaN
5684,Mauritius,2018,Total,October,Data tabulated by year of registration,"Final figure, complete",2019.0,888.0,24
8028,Serbia,2013,Total,September,Data tabulated by year of registration,"Final figure, complete",2015.0,7500.0,34


### b. Programmatic assessment

In [1299]:
# List of countries that are avaoilable in John Hopkins Dataset
df_JHU_Recovered['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Brazil', 'Brunei',
       'Bulgaria', 'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Canada', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Colombia', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Diamond Princess',
       'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia',
       'Germany', 'Ghana', 'Grenada', 'Greece', 'Guatemala', 'Guinea',
       'Guinea-Bissau', 'Guyana', 'Haiti', 'H

In [1300]:
# List of countries that are avaoilable in John Hopkins Dataset
df_OWID_Covid['location'].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Anguilla', 'Albania', 'Andorra',
       'United Arab Emirates', 'Argentina', 'Armenia',
       'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan',
       'Burundi', 'Belgium', 'Benin', 'Bonaire Sint Eustatius and Saba',
       'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland', 'Chile',
       'China', "Cote d'Ivoire", 'Cameroon',
       'Democratic Republic of Congo', 'Congo', 'Colombia', 'Comoros',
       'Cape Verde', 'Costa Rica', 'Cuba', 'Curacao', 'Cayman Islands',
       'Cyprus', 'Czech Republic', 'Germany', 'Djibouti', 'Dominica',
       'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt',
       'Eritrea', 'Western Sahara', 'Spain', 'Estonia', 'Ethiopia',
       'Finland', 'Fiji', 'Falkland Islands', 'France',

In [1301]:
# Available variables in dataset
list(df_OWID_Covid)

['iso_code',
 'location',
 'date',
 'total_cases',
 'new_cases',
 'total_deaths',
 'new_deaths',
 'total_cases_per_million',
 'new_cases_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'total_tests',
 'new_tests',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'tests_units',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'extreme_poverty',
 'cvd_death_rate',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'handwashing_facilities',
 'hospital_beds_per_100k']

In [1302]:
df_OWID_Covid.query('location == "Germany" and date == "2020-05-13"')

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
4244,DEU,Germany,2020-05-13,171306,798,7634,101,2044.616,9.524,91.115,...,21.453,15.957,45229.245,NaN,156.139,8.31,28.2,33.1,NaN,8.0


In [1303]:
df_Check = df_JHU_Confirmed.copy()
df_Check.rename(columns={'Country/Region': 'country'}, inplace=True)
df_Check.query('country == "Germany"')

,Province/State,country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
120,NaN,Germany,51.0,9.0,0,0,0,0,0,1,...,169430,170588,171324,171879,172576,173171,174098,174478,175233,175752


In [1304]:
# Check for countries which are referred to by different names in different dataframes
c_df_JHU_Fatality =  df_JHU_Fatality['Country/Region'].unique()
c_df_JHU_Confirmed = df_JHU_Confirmed['Country/Region'].unique()
c_df_JHU_Recovered = df_JHU_Recovered['Country/Region'].unique()
c_df_JHU_Countries = df_JHU_Countries['Country/Region'].unique()
c_df_OWID_Covid = df_OWID_Covid['location'].unique()
c_df_OWID_Testing = df_OWID_Testing['Entity'].unique()
c_df_OWID_Countries = df_OWID_Countries['location'].unique()
c_df_WIKI_ICU = df_WIKI_ICU['countryname'].unique()
c_df_UN_births = df_UN_births['Country or Area'].unique()
c_df_UN_deaths = df_UN_deaths['Country or Area'].unique()

all_country_names = list(c_df_JHU_Fatality) + list(c_df_JHU_Confirmed) + list(c_df_JHU_Recovered) + list(c_df_JHU_Countries) + list(c_df_OWID_Covid) + list(c_df_OWID_Testing) + list(c_df_OWID_Countries) + list(c_df_WIKI_ICU) + list(c_df_UN_births) + list(c_df_UN_deaths)

all_country_names = pd.Series(all_country_names).unique()

In [1305]:
all_country_names

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Diamond Princess', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India

### Findings, which contradict requirements:

#### Quality Observations:
- Validity: Some observations/rows in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' contain the values for a region, for example Australia appears multiple times in column country as the observations are per region.
- Consistency: Data about Covid-19 cases differs slightly between John Hopkins and OWID, data which is available in both datasets will be kept only from John Hopkins

#### Tidiness Observations:
- The data of 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' should be one observational unit 'df_covid' with columns 'country', 'date', 'recovered', 'confirmed', 'fatal' and 'date' beeing of type datetime.
- Column 'Country/Region' should only contain countries, therefore column name should by 'country', same for OWID data.
- Columns 'Province/State', 'Lat' and 'Long' are not necessary in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality'
- Data for countries, which are not of interested is not needed in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality', 'df_JHU_Countries'
- In the df_OWID_Covid_clean dataframes there is covid-related data where the variation frequency is daily and there is data not directly covid-related where data variation frequency is monthly or even constant for . Thus, there should be three observational units, df_covid for covid-related data and daily observations, df_OWID_country.




- df_OWID_Countries convert datatype population to integer
- df_OWID_Countries drop 'countriesAndTerritories', 'population_year'
- df_ESTAT_census make columns from values in n_person
- df_ESTAT_census replace 'Germany (until 1990 former territory of the FRG)' with 'Germany'
- df_ESTAT_census drop country 'Bulgaria'
- df_ESTAT_census drop '4 persons', '5 persons', '6 persons or more'

-  merge df_OWID_Countries with df_country

<a id='clean'></a>
## 4. Clean data

In [1306]:
# Create copies for cleaning process to preserve original dataframes
df_JHU_Fatality_clean = df_JHU_Fatality.copy()
df_JHU_Confirmed_clean = df_JHU_Confirmed.copy()
df_JHU_Recovered_clean = df_JHU_Recovered.copy()
df_JHU_Countries_clean = df_JHU_Countries.copy()
df_OWID_Covid_clean = df_OWID_Covid.copy()
df_OWID_Testing_clean = df_OWID_Testing.copy()
df_OWID_Countries_clean = df_OWID_Countries.copy()
df_ESTAT_census_clean = df_ESTAT_census.copy()
df_WIKI_ICU_clean = df_WIKI_ICU.copy()
df_UN_births_clean = df_UN_births.copy()
df_UN_deaths_clean = df_UN_deaths.copy()

### Issue 1:
#### Observe:
-  Tidiness: Columns 'Province/State', 'Lat' and 'Long' are not necessary in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality'

#### Define:
- Drop columns 'Province/State', 'Lat' and 'Long'

#### Code:

In [1307]:
# Drop variables which are only necessary for retweets
df_JHU_Fatality_clean.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)
df_JHU_Confirmed_clean.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)
df_JHU_Recovered_clean.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)

#### Test:

In [1308]:
# Check if columnns 'Province/State', 'Lat' and 'Long' dropped
df_JHU_Fatality_clean.head(1)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,106,109,115,120,122,127,132,136,153,168


In [1309]:
# Check if columnns 'Province/State', 'Lat' and 'Long' dropped
df_JHU_Confirmed_clean.head(1)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,3563,3778,4033,4402,4687,4963,5226,5639,6053,6402


In [1310]:
# Check if columnns 'Province/State', 'Lat' and 'Long' dropped
df_JHU_Recovered_clean.head(1)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,468,472,502,558,558,610,648,691,745,745


### Issue 2:
#### Observe:
- Tidiness: Column 'Country/Region' should only contain countries, therefore column name should by 'Country'.

#### Define:
- Rename column 'Country/Region' to 'country'

#### Code:

In [1311]:
# Rename coloumn inplace to identic primary key
df_JHU_Fatality_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_JHU_Confirmed_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_JHU_Recovered_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_JHU_Countries_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_OWID_Covid_clean.rename(columns={'location': 'country'}, inplace=True)
df_OWID_Countries_clean.rename(columns={'location': 'country'}, inplace=True)

#### Test:

In [1312]:
assert df_JHU_Fatality_clean.country.any()

In [1313]:
assert df_JHU_Confirmed_clean.country.any()

In [1314]:
assert df_JHU_Recovered_clean.country.any()

In [1315]:
assert df_JHU_Countries_clean.country.any()

In [1316]:
assert df_OWID_Covid_clean.country.any()

In [1317]:
assert df_OWID_Countries_clean.country.any()

### Issue 3:
#### Observe:
- Tidiness: In the df_OWID_Covid_clean dataframes there is covid-related data where the variation frequency is daily and there is data not directly covid-related where data variation frequency is monthly or even constant for . Thus, there should be three observational units, df_covid for covid-related data and daily observations, df_OWID_country.

#### Define
- Create new dataframe df_OWID_Countries with columns 'iso_code', 'location', 'population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'diabetes_prevalence', 'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_100k'.

#### Code:

In [1318]:
df_OWID_Countries = df_OWID_Covid_clean.copy()
df_OWID_Countries.drop([ 'date',
                         'total_cases',
                         'new_cases',
                         'total_deaths',
                         'new_deaths',
                         'total_cases_per_million',
                         'new_cases_per_million',
                         'total_deaths_per_million',
                         'new_deaths_per_million',
                         'total_tests',
                         'new_tests',
                         'total_tests_per_thousand',
                         'new_tests_per_thousand',
                         'tests_units',
                         'cvd_death_rate',
                         'handwashing_facilities',
                         'extreme_poverty'], axis=1, inplace=True)
df_OWID_Countries = df_OWID_Countries.drop_duplicates()

#subset=['A', 'C'], 

#### Test:

In [1319]:
df_OWID_Countries.country.value_counts().head(3)

Mexico    1
Iran      1
Guinea    1
Name: country, dtype: int64

In [1320]:
list(df_OWID_Countries)

['iso_code',
 'country',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'hospital_beds_per_100k']

### Issue 4:
#### Observe:
- Validity: Some observations/rows in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' contain the values for a region, for example Australia appears multiple times in column country as the observations are per region.

#### Define: 
- Sum values of rows with same entry in column country by using groupby

#### Code:

In [1321]:
# Groupby and sum
df_JHU_Fatality_clean = df_JHU_Fatality_clean.groupby(['country'], as_index=False).sum()
df_JHU_Confirmed_clean = df_JHU_Confirmed_clean.groupby(['country'], as_index=False).sum()
df_JHU_Recovered_clean = df_JHU_Recovered_clean.groupby(['country'], as_index=False).sum()

#### Test:

In [1322]:
df_JHU_Fatality_clean.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
183    False
184    False
185    False
186    False
187    False
Length: 188, dtype: bool

### Issue 5:
#### Observe:
- Tidiness: The data of 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' should be one observational unit 'df_covid' with columns 'country', 'date', 'recovered', 'confirmed', 'fatal' and 'date' beeing of type datetime.

#### Define:
- Melt date columns to one column 'date', transform date to type datetime and merge the three dataframes to ones dataframe 'df_covid' with sorted date values.

#### Code:

In [1323]:
# Melt each dataframe so that results in columns: country,
df_JHU_Fatality_clean = pd.melt(df_JHU_Fatality_clean, id_vars = ['country'], var_name='date', value_name='fatal')
df_JHU_Confirmed_clean = pd.melt(df_JHU_Confirmed_clean, id_vars = ['country'], var_name='date', value_name='confirmed')
df_JHU_Recovered_clean = pd.melt(df_JHU_Recovered_clean, id_vars = ['country'], var_name='date', value_name='recovered')

In [1324]:
# Convert new columns date to datetime
df_JHU_Fatality_clean.date=pd.to_datetime(df_JHU_Fatality_clean.date)
df_JHU_Confirmed_clean.date=pd.to_datetime(df_JHU_Confirmed_clean.date)
df_JHU_Recovered_clean.date=pd.to_datetime(df_JHU_Recovered_clean.date)

In [1325]:
# Merge three covid dataframes to one
df_covid = pd.merge(df_JHU_Fatality_clean, df_JHU_Confirmed_clean, on=['country','date'])
df_covid = pd.merge(df_covid, df_JHU_Recovered_clean, on=['country','date'])

In [1326]:
# Sort date values by date
df_covid = df_covid.sort_values(by='date', ascending=True)

#### Test:

In [1327]:
list(df_covid)

['country', 'date', 'fatal', 'confirmed', 'recovered']

In [1328]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21808 entries, 0 to 21807
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   country    21808 non-null  object        
 1   date       21808 non-null  datetime64[ns]
 2   fatal      21808 non-null  int64         
 3   confirmed  21808 non-null  int64         
 4   recovered  21808 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 1022.2+ KB


In [1329]:
df_covid

,country,date,fatal,confirmed,recovered
0,Afghanistan,2020-01-22,0,0,0
120,Namibia,2020-01-22,0,0,0
121,Nepal,2020-01-22,0,0,0
122,Netherlands,2020-01-22,0,0,0
123,New Zealand,2020-01-22,0,0,0
...,...,...,...,...,...
21684,Gambia,2020-05-16,1,23,12
21685,Georgia,2020-05-16,12,683,419
21686,Germany,2020-05-16,7938,175752,152600
21677,Estonia,2020-05-16,63,1770,934


### Issue 6:
#### Observe:
- Consistency: Some countries are referred to with varying names, for example 'US' and 'United Stats'. Other names are not valid.

#### Define:
- Clean names of all dataframes by extracting country names with regex expressions and renaming countries.

#### Code

In [1330]:
# Rename coloumn inplace
df_covid['country'].replace({'US': 'United States', 'Taiwan*': 'Taiwan'}, inplace=True)
df_JHU_Countries_clean['country'].replace({'US': 'United States', 'Taiwan*': 'Taiwan'}, inplace=True)

In [1331]:
# Extract country name from column 'Entity' via regex
df_OWID_Testing_clean['country'] = df_OWID_Testing_clean.Entity.str.extract(
    '([A-Z][a-z]{0,20}( [A-Z][a-z]{0,20})?)', expand=True)[0]

# Drop column from which country name was extracted
df_OWID_Testing_clean.drop(['Entity'], axis=1, inplace=True)

In [1332]:
# Extract country name from column 'Country or Area' via regex
df_UN_births_clean['country'] = df_UN_births_clean['Country or Area'].str.extract(
    '([A-Z][a-z]{0,20}( [A-Z][a-z]{0,20})?)', expand=True)[0]

df_UN_deaths_clean['country'] = df_UN_deaths_clean['Country or Area'].str.extract(
    '([A-Z][a-z]{0,20}( [A-Z][a-z]{0,20})?)', expand=True)[0]

# Drop column from which country name was extracted
df_UN_births_clean.drop(['Country or Area'], axis=1, inplace=True)
df_UN_deaths_clean.drop(['Country or Area'], axis=1, inplace=True)

In [1333]:
# Extract country name from column 'Entity' via regex
df_WIKI_ICU_clean['country'] = df_WIKI_ICU_clean.countryname.str.extract(
    '([A-Z][a-z]{0,20}( [A-Z][a-z]{0,20})?)', expand=True)[0]

# Drop column from which country name was extracted
df_WIKI_ICU_clean.drop(['countryname'], axis=1, inplace=True)

In [1334]:
# Rename coloumn inplace where different names refer to same country
df_UN_births_clean['country'].replace({'Russian Federation': 'Russia', 
                                       'British Virgin': 'British Virgin Islands',
                                       'Bosnia': 'Bosnia and Herzegovina',
                                       'Trinidad': 'Trinidad and Tobago',
                                       'Turks': 'Turks and Caicos Islands'}, inplace=True)

df_UN_deaths_clean['country'].replace({'Russian Federation': 'Russia', 
                                       'British Virgin': 'British Virgin Islands', 
                                       'Bosnia': 'Bosnia and Herzegovina',
                                       'Trinidad': 'Trinidad and Tobago',
                                       'Turks': 'Turks and Caicos Islands'}, inplace=True)


df_OWID_Countries_clean['country'].replace({'Macedonia': 'North Macedonia'}, inplace=True)


In [1335]:
# Create array with countries of interest
countries =['Afghanistan',
             'Albania',
             'Algeria',
             'American Samoa',
             'Andorra',
             'Angola',
             'Anguilla',
             'Antigua and Barbuda',
             'Argentina',
             'Armenia',
             'Aruba',
             'Australia',
             'Austria',
             'Azerbaijan',
             'Bahamas',
             'Bahrain',
             'Bangladesh',
             'Barbados',
             'Belarus',
             'Belgium',
             'Belize',
             'Benin',
             'Bermuda',
             'Bhutan',
             'Bolivia',
             'Bonaire Sint Eustatius and Saba',
             'Bosnia and Herzegovina',
             'Botswana',
             'Brazil',
             'British Virgin Islands',
             'Brunei',
             'Bulgaria',
             'Burkina Faso',
             'Burma',
             'Burundi',
             'Cabo Verde',
             'Cambodia',
             'Cameroon',
             'Canada',
             'Cape Verde',
             'Cayman Islands',
             'Central African Republic',
             'Chad',
             'Chile',
             'China',
             'Colombia',
             'Comoros',
             'Congo',
             'Cook Islands',
             'Costa Rica',
             "Cote d'Ivoire",
             'Croatia',
             'Cuba',
             'Cura',
             'Curacao',
             'Cyprus',
             'Czech Republic',
             'Czechia',
             'Denmark',
             'Diamond Princess',
             'Djibouti',
             'Dominica',
             'Dominican Republic',
             'Ecuador',
             'Egypt',
             'El Salvador',
             'Equatorial Guinea',
             'Eritrea',
             'Estonia',
             'Eswatini',
             'Ethiopia',
             'Faeroe Islands',
             'Falkland Islands',
             'Faroe Islands',
             'Fiji',
             'Finland',
             'France',
             'French Polynesia',
             'Gabon',
             'Gambia',
             'Georgia',
             'Germany',
             'Ghana',
             'Gibraltar',
             'Greece',
             'Greenland',
             'Grenada',
             'Guam',
             'Guatemala',
             'Guernsey',
             'Guinea',
             'Guinea-Bissau',
             'Guyana',
             'Haiti',
             'Holy See',
             'Honduras',
             'Hong Kong',
             'Hungary',
             'Iceland',
             'India',
             'Indonesia',
             'International',
             'Iran',
             'Iraq',
             'Ireland',
             'Islands',
             'Isle',
             'Isle of Man',
             'Israel',
             'Italy',
             'Jamaica',
             'Japan',
             'Jersey',
             'Jordan',
             'Kazakhstan',
             'Kenya',
             'Korea, South',
             'Kosovo',
             'Kuwait',
             'Kyrgyzstan',
             'Laos',
             'Latvia',
             'Lebanon',
             'Lesotho',
             'Liberia',
             'Libya',
             'Liechtenstein',
             'Lithuania',
             'Luxembourg',
             'MS Zaandam',
             'Madagascar',
             'Malawi',
             'Malaysia',
             'Maldives',
             'Mali',
             'Malta',
             'Mauritania',
             'Mauritius',
             'Mexico',
             'Moldova',
             'Monaco',
             'Mongolia',
             'Montenegro',
             'Montserrat',
             'Morocco',
             'Mozambique',
             'Myanmar',
             'Namibia',
             'Nepal',
             'Netherlands',
             'New Caledonia',
             'New Zealand',
             'Nicaragua',
             'Niger',
             'Nigeria',
             'North Macedonia',
             'Northern Mariana Islands',
             'Norway',
             'Oman',
             'Pakistan',
             'Palau',
             'Palestine',
             'Panama',
             'Papua New Guinea',
             'Paraguay',
             'Peru',
             'Philippines',
             'Poland',
             'Portugal',
             'Puerto Rico',
             'Qatar',
             'Republic',
             'Romania',
             'Russia',
             'Rwanda',
             'Saint Helena',
             'Saint Kitts and Nevis',
             'Saint Lucia',
             'Saint Vincent',
             'Saint Vincent and the Grenadines',
             'San Marino',
             'Sao Tome and Principe',
             'Saudi Arabia',
             'Senegal',
             'Serbia',
             'Seychelles',
             'Sierra Leone',
             'Singapore',
             'Sint Maarten (Dutch part)',
             'Slovakia',
             'Slovenia',
             'Somalia',
             'South Africa',
             'South Korea',
             'South Sudan',
             'Spain',
             'Sri Lanka',
             'Sudan',
             'Suriname',
             'Swaziland',
             'Sweden',
             'Switzerland',
             'Syria',
             'Taiwan',
             'Tajikistan',
             'Tanzania',
             'Thailand',
             'Timor',
             'Timor-Leste',
             'Togo',
             'Trinidad and Tobago',
             'Tunisia',
             'Turkey',
             'Turks and Caicos Islands',
             'Uganda',
             'Ukraine',
             'United Arab Emirates',
             'United Kingdom',
             'United States',
             'United States Virgin Islands',
             'Uruguay',
             'Uzbekistan',
             'Vatican',
             'Venezuela',
             'Vietnam',
             'West Bank and Gaza',
             'Western Sahara',
             'World',
             'Yemen',
             'Zambia',
             'Zimbabwe']

In [1336]:
# Only keep Countries of interes
df_covid = df_covid[df_covid['country'].isin(countries)]
df_JHU_Countries_clean = df_JHU_Countries_clean[df_JHU_Countries_clean['country'].isin(countries)]
df_OWID_Covid_clean = df_OWID_Covid_clean[df_OWID_Covid_clean['country'].isin(countries)]
df_OWID_Testing_clean = df_OWID_Testing_clean[df_OWID_Testing_clean['country'].isin(countries)]
df_OWID_Countries_clean = df_OWID_Countries_clean[df_OWID_Countries_clean['country'].isin(countries)]
df_WIKI_ICU_clean = df_WIKI_ICU_clean[df_WIKI_ICU_clean['country'].isin(countries)]
df_UN_births_clean = df_UN_births_clean[df_UN_births_clean['country'].isin(countries)]
df_UN_deaths_clean = df_UN_deaths_clean[df_UN_deaths_clean['country'].isin(countries)]

#### Test:

In [1348]:
# Visually check countries of all dataframes

print(list(pd.Series(df_covid['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_JHU_Countries_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_OWID_Covid_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_OWID_Testing_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_OWID_Countries_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_WIKI_ICU_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_UN_births_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_UN_deaths_clean['country'].unique()).sort_values()))
print('\n\n')



['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', '

In [1338]:
# Check for countries which are referred to by different names in different dataframes
c_df_covid = df_covid['country'].unique()
c_df_JHU_Countries_clean = df_JHU_Countries_clean['country'].unique()
c_df_OWID_Covid_clean = df_OWID_Covid_clean['country'].unique()
c_df_OWID_Testing_clean = df_OWID_Testing_clean['country'].unique()
c_df_OWID_Countries_clean = df_OWID_Countries_clean['country'].unique()
c_df_WIKI_ICU_clean = df_WIKI_ICU_clean['country'].unique()
c_df_UN_births_clean = df_UN_births_clean['country'].unique()
c_df_UN_deaths_clean = df_UN_deaths_clean['country'].unique()

all_country_names = list(c_df_covid) + list(c_df_JHU_Countries_clean) + list(c_df_OWID_Covid_clean) + list(c_df_OWID_Testing_clean) + list(c_df_OWID_Countries_clean) + list(c_df_WIKI_ICU_clean) + list(c_df_UN_births_clean) + list(c_df_UN_deaths_clean)

# countries = pd.Series(all_country_names).sort_values()

# Create alphabetically sorted list of all countries in all dataframes
list(pd.Series(pd.Series(all_country_names).unique()).sort_values())

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bonaire Sint Eustatius and Saba',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cura',
 'Curacao',
 'Cyprus',
 'Czech Republic',
 'Czechia',
 'Denmark',
 'Diamond Princess',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'E

In [1339]:
assert len(pd.Series(all_country_names).unique()) == len(countries)

True

In [1347]:
list(df_OWID_Testing_clean)

['ISO code',
 'Date',
 'Source URL',
 'Source label',
 'Notes',
 'Number of observations',
 'Cumulative total',
 'Cumulative total per thousand',
 'Daily change in cumulative total',
 'Daily change in cumulative total per thousand',
 '3-day rolling mean daily change',
 '3-day rolling mean daily change per thousand',
 '7-day rolling mean daily change',
 '7-day rolling mean daily change per thousand',
 'General source label',
 'General source URL',
 'Short description',
 'Detailed description',
 'country']


- df_OWID_Testing_clean drop columns 'source URL', 'Source label', 'Notes', 'Number of observations', 'Daily change in cumulative total', 'Daily change in cumulative total per thousand', '3-day rolling mean daily change', '3-day rolling mean daily change per thousand', '7-day rolling mean daily change', '7-day rolling mean daily change per thousand','General source label', 'General source URL', 'Short description', 'Detailed description'
- df_OWID_Testing_clean Either cut per regex country name from 'Entity' and rename country or join country name from other df

<a id='store'></a>
## 5. Store clean data

In [1346]:
# Store cleaned dataset to csv
df_covid.to_csv('covid_master.csv', encoding='utf-8')